In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.tree import DecisionTreeClassifier

In [3]:
df = pd.read_csv('titanic.csv')

In [4]:
df.sample()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
681,682,1,1,"Hassab, Mr. Hammad",male,27.0,0,0,PC 17572,76.7292,D49,C


In [6]:
df.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)

In [7]:
df.sample()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
840,0,3,male,20.0,0,0,7.925,S


In [8]:
#step1 -->train/test/split
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['Survived']),df['Survived'],test_size=0.2,random_state=43)

In [9]:
X_train.shape

(712, 7)

In [10]:
X_train.head(2)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
826,3,male,NaN,0,0,56.4958,S
838,3,male,32.0,0,0,56.4958,S


In [11]:
#imputation Transformer
trf1 = ColumnTransformer([
    ('impute_age',SimpleImputer(),[2]), #always a better practice to go through index number
    ('impute_embarked',SimpleImputer(strategy='most_frequent'),[6])
],remainder='passthrough')

In [12]:
#onehotEncoding
trf2 = ColumnTransformer([
    ('ohe_sex_embarked',OneHotEncoder(sparse=False,handle_unknown='ignore'),[1,6])
],remainder='passthrough')

In [13]:
#scaling
trf3 = ColumnTransformer([
    ('scale',MinMaxScaler(),slice(1,10))#we will get 10 columns after column transfer
])

In [14]:
#feature Selection
trf4 = SelectKBest(score_func=chi2,k=6) #k can be anything...select best out of all

In [15]:
#train this model
trf5 = DecisionTreeClassifier()

# Create Pipeline

In [19]:
#display pipeline
from sklearn import set_config
set_config(display='diagram')

In [20]:
pipe = Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3),
    ('trf4',trf4),
    ('trf5',trf5)
])

# Pipeline vs make_pipeline

## Pipeline requires naming of steps, make_pipeline does not.
## Same applies to ColumnTransformer vs make_column_transformer

In [21]:
#alternate syntax
pipe1 = make_pipeline(trf1,trf2,trf3,trf4,trf5)

In [22]:
pipe


Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(1, 10, None))])),
                ('trf4',
                 SelectKBest(k=6,
                             score_func=<function chi2 at 0x7f20358cb5e0>)),
                ('trf5', DecisionTreeClassifier())])

In [23]:
pipe1

Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('columntransformer-2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('columntransformer-3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(1, 10, None))])),
                ('selectkbest',
                 SelectKBest(k=6,
                             score_func=<function chi2 at 0x7f20358cb5e0>)),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

### pipeline is better as it is easy to debug

In [24]:
#train
pipe.fit(X_train,y_train)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(1, 10, None))])),
                ('trf4',
                 SelectKBest(k=6,
                             score_func=<function chi2 at 0x7f20358cb5e0>)),
                ('trf5', DecisionTreeClassifier())])

# Explore the Pipelines

In [25]:
pipe.named_steps

{'trf1': ColumnTransformer(remainder='passthrough',
                   transformers=[('impute_age', SimpleImputer(), [2]),
                                 ('impute_embarked',
                                  SimpleImputer(strategy='most_frequent'),
                                  [6])]),
 'trf2': ColumnTransformer(remainder='passthrough',
                   transformers=[('ohe_sex_embarked',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse=False),
                                  [1, 6])]),
 'trf3': ColumnTransformer(transformers=[('scale', MinMaxScaler(), slice(1, 10, None))]),
 'trf4': SelectKBest(k=6, score_func=<function chi2 at 0x7f20358cb5e0>),
 'trf5': DecisionTreeClassifier()}

In [28]:
pipe.named_steps['trf1'].transformers_ #to find the mean value

[('impute_age', SimpleImputer(), [2]),
 ('impute_embarked', SimpleImputer(strategy='most_frequent'), [6]),
 ('remainder', 'passthrough', [0, 1, 3, 4, 5])]

In [34]:
pipe.named_steps['trf1'].transformers_[0][1].statistics_

array([30.16725217])

pipe.named_steps['trf1'].transformers_[1][1].statistics_

In [40]:
#predict
y_pred = pipe.predict(X_test)

In [41]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.6145251396648045

## Cross validation using pipeline

In [42]:
#cross validation using cross_val_scores
from sklearn.model_selection import cross_val_score
cross_val_score(pipe,X_train,y_train,cv=5,scoring='accuracy').mean()

0.6081158278341376

## GridSearch using pipeline

In [43]:
#gridSearch
params = {
    'trf5__max_depth':[1,2,3,4,5,None]
} 

In [44]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(pipe,params,cv=5,scoring='accuracy')
grid.fit(X_train,y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('trf1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('impute_age',
                                                                         SimpleImputer(),
                                                                         [2]),
                                                                        ('impute_embarked',
                                                                         SimpleImputer(strategy='most_frequent'),
                                                                         [6])])),
                                       ('trf2',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('ohe_sex_embarked',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse=False),
                                                                         [1,
                                                                          6])])),
                                       ('trf3',
                                        ColumnTransformer(transformers=[('scale',
                                                                         MinMaxScaler(),
                                                                         slice(1, 10, None))])),
                                       ('trf4',
                                        SelectKBest(k=6,
                                                    score_func=<function chi2 at 0x7f20358cb5e0>)),
                                       ('trf5', DecisionTreeClassifier())]),
             param_grid={'trf5__max_depth': [1, 2, 3, 4, 5, None]},
             scoring='accuracy')

In [45]:
grid.best_score_

0.6081256771397616

In [46]:
grid.best_params_

{'trf5__max_depth': 1}

## Exporting the pipiline

In [47]:
import pickle
pickle.dump(pipe,open('pipe.pkl','wb'))